## The script below creates a Supply Chain BOM based on Enginerring BOM using data from a out-dated supply chain BOM. It also enables the supply chain BOM to sync with the Engineering BOM. Developed by yifei.li@byton.com

In [1]:
import pandas as pd
import numpy as np
import xlrd, openpyxl
import csv, datetime

## Load EBOM, Chi's Supply Chain BOM and Shaolong's BOM

In [2]:
#Engineering BOM (EBOM). Updated weekly and has the most up-to-date Part Number and Revision. 
#Supply Chain BOM is based on EBOM with additional sourcing and supplier quality information.

EBOM = pd.read_excel("/home/yifei/Documents/BOM_tool/EBOM 8.23.18.xlsx")
# new_header = EBOM.iloc[0]
# EBOM = EBOM[1:]
# EBOM.columns = new_header
EBOM.columns = EBOM.columns.str.replace("\(R\)\ ", "") # trim (R) away from the header
print("Engineering VP BOM Shape: ",EBOM.shape)
EBOM.head(5)
# print(EBOM.columns.values)
# EBOM.shape





Engineering VP BOM Shape:  (3164, 14)


,Identifier,Title,Revision,Description,QTY,UOM,Purchased Part Type,Maturity,Part Type,System,SubSystem,Legacy Part Number,Legacy Part Revision,Configuration
0,0,AA-ZZ000001NN-03,A.1,PHTM EC001 VP,1,EA EACH,NaN,In Work,Phantom,Z Vehicle Top Level Cfg,Z Z NA,NaN,NaN,NaN
1,1,AA-AZ000002NN-03,A.1,PHTM BIW VP,1,EA EACH,NaN,In Work,Phantom,A BIW,A Z NA,NaN,NaN,NaN
2,1|1,AA-AF000002NN-03,A.1,PHTM BIW ROOF STRUCTURE VP,1,EA EACH,NaN,In Work,Phantom,A BIW,A F Roof Structure,NaN,NaN,NaN
3,1|1|1,AA-AF003414NN-00,A.2,FULL_ROOF,1,EA EACH,NaN,In Work,NaN,A BIW,A F Roof Structure,EC001_02_521_000,NaN,NaN
4,1|1|1|1,AA-AF003415NN-00,A.1,FULL ROOF,1,NaN,NaN,In Work,NaN,A BIW,A F Roof Structure,EC001_02_501_101_PRT_019,19,NaN


In [3]:
print(EBOM.columns.values)

['Identifier' 'Title' 'Revision' 'Description' 'QTY' 'UOM'
 'Purchased Part Type' 'Maturity' 'Part Type' 'System' 'SubSystem'
 'Legacy Part Number' 'Legacy Part Revision' 'Configuration']


Removing Phantom part and Fasterners and filter only "Buy" part leaves us 1461 Unique PNs

In [4]:
EBOM_filtered = EBOM[(EBOM["Purchased Part Type"]=="Buy") & (EBOM["Part Type"]!="Phantom") & (EBOM["System"]!="Y Fasteners")]
print(EBOM_filtered.shape)


(1461, 14)


In [5]:
#Chi's SCBOM. We need the column names from Chi's SCBOM
SCBOM_chi = pd.read_excel("/home/yifei/Documents/BOM_tool/SCBOM.xlsx")
print("Supply Chain BOM Shape: ",SCBOM_chi.shape)
SCBOM_chi.head(20)


Supply Chain BOM Shape:  (6253, 47)


,Key,Identifier,PartNumber,PartRevision,LegacyPartNumber,LegacyPartRevision,System,SubSystem,Description,QTY,...,LastPickupDate,SC_PackagingeProposalApproved,PartStatus,PartActive,Maturity,RevisionComment,Configuration,EngineeringReleaseEstimate,EngineeringReleaseActual,AssemblyStation
0,AA-YD004238NN-00A.1EC001_04_999_0059VP,NaN,AA-YD004238NN-00,A.1,EC001_04_999_0059,NaN,EXTERIOR -STANDARD PARTS,REAR BUMPER,FLANGE NUT M6,6.0,...,NaT,False,Removed,Inactive,NaN,Set2 Count = 1 Set3 Count = 1,NaN,NaN,NaN,NaN
1,(2)7052134730A.1VP,11|3|8|1|8,(2)7052134730,A.1,NaN,NaN,NaN,NaN,NaN,6.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN
2,(2)7282-6443-40_Dup4A.1VP,11|3|8|1|18,(2)7282-6443-40_Dup4,A.1,NaN,NaN,NaN,NaN,NaN,3.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN
3,(2)7283-3441-40_prd-BYT-00606114A.1VP,11|3|17|2|5,(2)7283-3441-40_prd-BYT-00606114,A.1,NaN,NaN,NaN,NaN,NaN,1.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN
4,_DMU7282-6453-60A.1VP,11|3|17|2|10,_DMU7282-6453-60,A.1,NaN,NaN,NaN,NaN,NaN,1.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN
5,_DMU7283645440.1A.1VP,11|3|17|2|7,_DMU7283645440.1,A.1,NaN,NaN,NaN,NaN,NaN,1.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN
6,_FLT2471952XA.1VP,4|4|7|1,_FLT2471952X,A.1,NaN,NaN,NaN,NaN,NaN,1.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN
7,_FLT2471954XA.1VP,4|4|8|1,_FLT2471954X,A.1,NaN,NaN,NaN,NaN,NaN,1.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN
8,_FLT2471955XA.1VP,4|4|9|1,_FLT2471955X,A.1,NaN,NaN,NaN,NaN,NaN,1.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN
9,_FLT2492062XA.1VP,4|4|3|1,_FLT2492062X,A.1,NaN,NaN,NaN,NaN,NaN,1.0,...,NaT,False,Latest Revision,Active,In Work,NaN,NaN,NaN,NaN,NaN


In [6]:
SCBOM_chi.columns

Index(['Key', 'Identifier', 'PartNumber', 'PartRevision', 'LegacyPartNumber',
       'LegacyPartRevision', 'System', 'SubSystem', 'Description', 'QTY',
       'Type', 'Engineer', 'Program', 'PurchasePartType', 'NominatedSupplier',
       'ConfirmedSupplier', 'GSM', 'RFQSent', 'RFQReceived', 'LOIReceived',
       'SourcingCouncilScheduledDate', 'SourcingCouncilAprovedDate', 'DSA',
       'GTC', 'AwardLetter', 'PurchaseRequistion', 'PurchaseOrder',
       'ManufacturingLT', 'EngineeringReleaseNeededBy', 'CreatedDate',
       'GSMComment', 'ModifiedDate', 'FirstPickupDate', 'CCCRequired',
       'PieceCost', 'CustomComplianceCompleted', 'SubCommodity',
       'LastPickupDate', 'SC_PackagingeProposalApproved', 'PartStatus',
       'PartActive', 'Maturity', 'RevisionComment', 'Configuration',
       'EngineeringReleaseEstimate', 'EngineeringReleaseActual',
       'AssemblyStation'],
      dtype='object')

In [7]:
PreAP_BOM_chi = SCBOM_chi[SCBOM_chi["Program"]=="PAP"]
print("Pre AP BOM shape: ", PreAP_BOM_chi.shape)
AP_BOM_chi = SCBOM_chi[SCBOM_chi["Program"]=="AP"]
print("AP BOM shape: ", AP_BOM_chi.shape)
VP_BOM_chi = SCBOM_chi[SCBOM_chi["Program"]=="VP"]
print("VP BOM shape: ", VP_BOM_chi.shape)

#Active VP BOM with latest part revision
VP_BOM_Activate_chi = VP_BOM_chi[(VP_BOM_chi["PartActive"]=="Active")]
print("VP Active BOM: ", VP_BOM_Activate_chi.shape)

VPM_BOM_Activate_chi_removePhantomPart = VP_BOM_Activate_chi[VP_BOM_Activate_chi["Type"]!="Phantom"]
print("removed phatom", VPM_BOM_Activate_chi_removePhantomPart.shape)

                                                   


Pre AP BOM shape:  (863, 47)
AP BOM shape:  (0, 47)
VP BOM shape:  (5390, 47)
VP Active BOM:  (5064, 47)
removed phatom (5000, 47)


In [8]:
#Load shaolong's SC BOM which has the most sourcing information updated by buyers and SQEs. 
SCBOM_shaolong = pd.read_excel("/home/yifei/Documents/BOM_tool/Shaolong BOM.xlsx", index_col=None)
# SCBOM_shaolong.reset_index()
SCBOM_shaolong

,No.,SYSTEM,Commodity Package,Count Commodity,Description,BYTON P/N,ENGINEER,NJ IPM,GSM,Senior GSM,...,Hard tool Completion,Home line,FTN,Grain,AAR,Run at rate,PV Completion,Phase 1 PSW,Phase 3 PSW,Comments
0,1900-01-01,LOW VOLTAGE SYSTEMS (E/E),12 V OUTLET,1.0,12V POWER OUTLET,AA-LE000742NN-00,noopur.divekar@byton.com,NaN,Jennie.Lu,Veronica.Park,...,2018-11-05 00:00:00,2018-11-12 00:00:00,,,,2019-03-11 00:00:00,2019-03-25 00:00:00,2019-06-21 00:00:00,,
1,1900-01-02,LOW VOLTAGE SYSTEMS (E/E),12 V OUTLET,0.0,CIGAR LIGHTER SOCKET,AA-LE001138NN-00,noopur.divekar@byton.com,NaN,Jennie.Lu,Veronica.Park,...,2018-11-05 00:00:00,2018-11-12 00:00:00,,,,2019-03-11 00:00:00,2019-03-25 00:00:00,2019-06-21 00:00:00,,
2,1900-01-03,LOW VOLTAGE SYSTEMS (E/E),12 V OUTLET,0.0,CIGAR LIGHTER POPOUT,AA-LE001137NN-00,noopur.divekar@byton.com,NaN,Jennie.Lu,Veronica.Park,...,2018-11-05 00:00:00,2018-11-12 00:00:00,,,,2019-03-11 00:00:00,2019-03-25 00:00:00,2019-06-21 00:00:00,,
3,1900-01-04,LOW VOLTAGE SYSTEMS (E/E),12V BATTERY,1.0,12V BATTERY,AA-LA000707NN-00,rockymark.juan@byton.com,NaN,Jennie.Lu,Veronica.Park,...,,,,,,,,,,"supplier should be ""C&D"""
4,1900-01-05,LOW VOLTAGE SYSTEMS (E/E),230 V outlet,1.0,230V HIGH VOLTAGE POWER OUTLET,AA-LE000746NN-00,noopur.divekar@byton.com,NaN,Jennie.Lu,Veronica.Park,...,2018-11-05 00:00:00,2018-11-12 00:00:00,,,,2019-03-11 00:00:00,2019-03-25 00:00:00,2019-06-21 00:00:00,,
5,1900-01-06,INTELLIGENT CAR ENTERTAINMENT (ICE),4 ARRAY MEMS MICROPHONE,1.0,4 ARRAY MEMS MICROPHONE,AA-ND000703FN-00,CHET.DIKEN@BYTON,Ryan.Dong,Ryan.Dong,Veronica.Park,...,,,,,,,,,,"Byton Inhouse design, production supplier not ..."
6,1900-01-07,CHASSIS,ACCELERATOR PEDAL,1.0,ACCELERATOR PEDAL LHD ASSY,AA-EB110100FN-00,gary.waite@ricardo.com,NaN,Tom.Peleg,Tom.Peleg,...,,,,,,,,,,Parts already SOP for Other customer
7,1900-01-08,INTELLIGENT CAR ENTERTAINMENT (ICE),ACOUSTIC VEHICLE ALERT SYSTEM,1.0,ACOUSTIC VEHICLE ALERT SYSTEM,AA-NH003864FN-03,MARKUS.KOCH@BYTON.COM,NaN,Veronica.Park,Veronica.Park,...,,,,,,,,,,Not sourced for SOP
8,1900-01-09,INTERIOR,ACU,1.0,AIRBAG CONTROL UNIT,AA-DJ000855NN-00,JOHN.SCHEICK@BYTON.COM,Kun Fang,Kun Fang,Shaolong.Huang,...,,,,,,,,,,No feedback form supplier
9,1900-01-10,EXTERIOR,AERO PNL D-PILLAR,1.0,ASSY AERO PNL D-PILLAR LH,AA-CF003398NL-00,Mark Bruderick,Mark Shen,Mark Shen,Harvey.Li,...,,,,,,,,,,


In [9]:
SCBOM_shaolong.columns.values

array(['No.', 'SYSTEM', 'Commodity Package', 'Count Commodity',
       'Description', 'BYTON P/N', 'ENGINEER', 'NJ IPM', 'GSM',
       'Senior GSM', 'VP\nSUPPLIER\n(Name)', 'Supplier Sourced?',
       'Supplier \nContact Person', 'Supplier \nContact E-mail',
       'Supplier Location', 'Over Sea Plant?',
       'FOB / FAC &Where?\n(if over sea)', 'Tier', 'Ship to',
       'GTC & Contract', 'SQE Priority?', 'Risky Esclation', 'JIS?',
       'Oversea Pick Up Location', 'Package cost confirmed?',
       'Transport Price quoted?', 'Daily Production Capability',
       'Order Leader Time', 'Mnimum Order Quantity', 'SQE', 'Priority',
       'Reason for Priority', 'APQP \nKICK OFF', 'Unnamed: 33',
       'Tooling\nLead time', 'Tool Kick off Plan', 'Hard tool Completion',
       'Home line', 'FTN', 'Grain', 'AAR', 'Run at rate', 'PV Completion',
       'Phase 1 PSW', 'Phase 3 PSW', 'Comments'], dtype=object)

In [10]:
print("Supply Chain BOM Shape: ",SCBOM_shaolong.shape)

Supply Chain BOM Shape:  (1516, 46)


## Step 0: Create a blank Supply Chain BOM using EBOM

In [11]:
# copy EBOM to the new SCBOM
print("EBOM shape is: ", EBOM.shape)
SCBOM = EBOM.copy()
print("SCBOM shape is: ", SCBOM.shape)
#specify SCBOM columns
SCBOM.columns.values

EBOM shape is:  (3164, 14)
SCBOM shape is:  (3164, 14)


array(['Identifier', 'Title', 'Revision', 'Description', 'QTY', 'UOM',
       'Purchased Part Type', 'Maturity', 'Part Type', 'System',
       'SubSystem', 'Legacy Part Number', 'Legacy Part Revision',
       'Configuration'], dtype=object)

In [12]:
ebom_column_header = EBOM.columns.values

sourcing_column_header = ["Commodity Package", "Engineer", "GSM", "Senior GSM", "RFQ Sent", "RFQ Received", 
                          "Sourcing Council Scheduled Date", "Sourcing Council Aproved Date", 
                          "Supplier Name", "Sourcing Status", "Supplier Category", "Supplier Contact Person",
                          "Supplier Contact Email", "Supplier Manufacturing Region", 
                          "Supplier Manufacturing Country", "Incoterm", "GTC", "LOI", "Purchase Contract", 
                          "JIS", "Tier", "Ship to", "Daily Production Capability","Order Leader Time ","MOQ",
                          "Purchae Requisition","Purchase Order", "First Pickup Date", "Last Pickup Date", 
                         ]

cost_column_header = ["Unit Price", "Tooling Cost", "ED&T","Oversea Pick Up Location","Package cost confirmed?",
                      "Transport Price quoted?","Total Investment","Currency",    
                     ]

logistics_column_header = ["HS code Export", "HS code Import", "Regulatory Requirements (Y/N)", "Logistic Planned (Y/N)", 
                          "Packaging Proposal Approved","EDI Setup"
                          ]

quality_column_header = ["SQE", "Priority", "APQP Kick Off", "APQP Status","Tooling Lead Time", 
                     "Tooling Kick Off Date", "Tooling Completion Date", "Home Line", "FTN", 
                     "Grain", "AAR", "Run At Rate", "PV Completion Date", "Phase 1 PSW Date", 
                     "Phase 3 PSW Date", "Comments"
                        ]

log_column_header = ["Engineering Release Estimtae", "Engineering Release Actual", "Assembly Station", 
                     "Maturity", "Configuration", "Part Creation Date", "Last Modified Date", "Part Status", 
                     "Part Active"
                    ]

column_header = sourcing_column_header + cost_column_header + logistics_column_header + quality_column_header + log_column_header

In [13]:
# add additional columns
def add_columns(df, columns):
    df2 = pd.DataFrame(columns=columns)
    df = df.append(df2, sort=False, verify_integrity=True)
    return df

In [14]:
SCBOM = add_columns(SCBOM, column_header)
SCBOM.shape

(3164, 80)

## Step 1: Update new SCBOM using Shaolong's BOM

In [15]:
df = pd.DataFrame({'BoolCol': [True, False, False, True, True], 'Name': [1,2,3,4,5]},
       index=[10,20,30,40,50])
idx = df.index[df['BoolCol']]
idx

Int64Index([10, 40, 50], dtype='int64')

In [16]:
df.loc[idx[0]]["Name"]

1

In [17]:
df

,BoolCol,Name
10,True,1
20,False,2
30,False,3
40,True,4
50,True,5


In [18]:
df.loc[(df["Name"]==2) & (df["BoolCol"]==False)].index.values

array([20])

In [19]:
df.loc[[10], ["Name"]] = df.loc[[20], 'Name'].values
df

,BoolCol,Name
10,True,2
20,False,2
30,False,3
40,True,4
50,True,5


In [20]:
# save dataframe to Excel
def save(df):
    #https://stackoverflow.com/questions/28837057/pandas-writing-an-excel-file-containing-unicode-illegalcharactererror
    df = df.applymap(lambda x: x.encode('unicode_escape').
                 decode('utf-8') if isinstance(x, str) else x)

    writer = pd.ExcelWriter('Supply Chain BOM.xlsx')
    df.to_excel(writer, sheet_name="Supply Chain BOM", na_rep="" )
    writer.save()

#     df.to_csv("Supply Chain BOM.csv",quoting=csv.QUOTE_NONE, escapechar="\\")

In [21]:
# search PN and Rev in shaolong's SCBOM
def search(PN):
    # shaolong's data doesn't have Rev.
#     df = SCBOM_shaolong.loc[(SCBOM_shaolong["BYTON P/N"]==PN) & (SCBOM_shaolong["BYTON REVISION"]==Rev)]
    df = SCBOM_shaolong.loc[(SCBOM_shaolong["BYTON P/N"]==PN)]
    index = df.index
    if (len(index)==1):
        return index
    else:
        #return the first index if duplicate PN found
        return index[0:1]
        

In [22]:
print(search("AA-AF003416NN-00"))

Int64Index([261], dtype='int64')


In [23]:
print(search("AA-YF000148NN-00"))

Int64Index([1105], dtype='int64')


In [24]:
def replace_bracket(data):
    value = str(data).replace('[', '').replace(']', '').replace('\'', '')
    if value == 'nan':
        return ""
    return value

def change_datetime_to_string(data):
    if data.size == 0:
        return ""
    elif (data == np.array([' '])):
        return ""
    elif data == "nan":
        return ""
    else:
        return data[0]
    
#     if data == np.array([' ']):
#         return ""
#     if data == "nan":
#         return ""
#     else:
#         return data[0].strftime('%m/%d/%Y')

In [25]:
# copy Shaolong's BOM info and paste into SCBOM
def copy_and_paste_row(df1, index1, df2, index2):
    #sourcing
    df1.loc[index1,["Engineer"]] = replace_bracket(df2.loc[index2,["ENGINEER"]].values)
    df1.loc[index1,["GSM"]] = replace_bracket(df2.loc[index2,["GSM"]].values)
    df1.loc[index1,["Senior GSM"]] = replace_bracket(df2.loc[index2,["Senior GSM"]].values)
    df1.loc[index1,["Supplier Name"]] = replace_bracket(df2.loc[index2,["VP\nSUPPLIER\n(Name)"]].values)
    df1.loc[index1,["Sourcing Status"]] = replace_bracket(df2.loc[index2,["Supplier Sourced?"]].values)
    df1.loc[index1,["Supplier Contact Person"]] = replace_bracket(df2.loc[index2,["Supplier \nContact Person"]].values)
    df1.loc[index1,["Supplier Contact Email"]] = replace_bracket(df2.loc[index2,["Supplier \nContact E-mail"]].values)
    df1.loc[index1,["Supplier Manufacturing Country"]] = replace_bracket(df2.loc[index2,["Supplier Location"]].values)
    df1.loc[index1,["GTC"]] = replace_bracket(df2.loc[index2,["GTC & Contract"]].values)

    
    #Logistics
    
    #SQE
    df1.loc[index1,["SQE"]] = replace_bracket(df2.loc[index2,["SQE"]].values)
    df1.loc[index1,["Priority"]] = replace_bracket(df2.loc[index2,["SQE Priority?"]].values)
    df1.loc[index1,["APQP Kick Off"]] = replace_bracket(df2.loc[index2,["APQP \nKICK OFF"]].values)
    df1.loc[index1,["APQP Status"]] = replace_bracket(df2.loc[index2,["Unnamed: 33"]].values)
    df1.loc[index1,["Tooling Lead Time"]] = replace_bracket(df2.loc[index2,["Tooling\nLead time"]].values)
    df1.loc[index1,["Tooling Kick Off Date"]] = change_datetime_to_string(df2.loc[index2,["Tool Kick off Plan"]].values)
    df1.loc[index1,["Tooling Completion Date"]] = change_datetime_to_string(df2.loc[index2,["Hard tool Completion"]].values)
    df1.loc[index1,["Home Line"]] = change_datetime_to_string(df2.loc[index2,["Home line"]].values)
    df1.loc[index1,["FTN"]] = replace_bracket(df2.loc[index2,["FTN"]].values)
    df1.loc[index1,["Grain"]] = replace_bracket(df2.loc[index2,["Grain"]].values)
    df1.loc[index1,["FTN"]] = replace_bracket(df2.loc[index2,["FTN"]].values)
    df1.loc[index1,["AAR"]] = replace_bracket(df2.loc[index2,["AAR"]].values)
    df1.loc[index1,["Run At Rate"]] = change_datetime_to_string(df2.loc[index2,["Run at rate"]].values)
    df1.loc[index1,["PV Completion Date"]] = change_datetime_to_string(df2.loc[index2,["PV Completion"]].values)
    df1.loc[index1,["Phase 1 PSW Date"]] = change_datetime_to_string(df2.loc[index2,["Phase 1 PSW"]].values)
    df1.loc[index1,["Phase 3 PSW Date"]] = change_datetime_to_string(df2.loc[index2,["Phase 3 PSW"]].values)
    df1.loc[index1,["Comments"]] = replace_bracket(df2.loc[index2,["Comments"]].values)
    
    
    #log
#   log_column_header = ["Engineering Release Estimtae", "Engineering Release Actual", "Assembly Station", 
#                      "Maturity", "Configuration", "Part Creation Date", "Last Modified Date", "Part Status", 
#                      "Part Active"]
    df1.loc[index1,["Part Status"]] = "Latest revision"
    df1.loc[index1,["Part Active"]] = "Active"
        
        
    return df1
    
    
    
     

In [26]:
print(str(SCBOM_shaolong.loc[1000, ["GSM"]].values))

['Joseph Zhu']


In [27]:
print(str(SCBOM_shaolong.loc[1000, ["GSM"]].values).replace('[', '').replace(']', '').replace('\'', ''))

Joseph Zhu


In [28]:
SCBOM_shaolong.loc[:, ["Tool Kick off Plan"]].values

array([[datetime.datetime(2018, 9, 13, 0, 0)],
       [datetime.datetime(2018, 9, 13, 0, 0)],
       [datetime.datetime(2018, 9, 13, 0, 0)],
       ...,
       [' '],
       [' '],
       [' ']], dtype=object)

In [29]:
SCBOM_shaolong.loc[41, ["Tool Kick off Plan"]].values[0]

datetime.datetime(2018, 9, 15, 0, 0)

In [30]:
# update SCBOM with inputs from Shaolong's BOM
for index, row in SCBOM.iterrows():
    #loop through EBOM by PN
    PN = row["Title"]
    Rev = row["Revision"]
    #find out if this PN and Rev exist in Shaolong's BOM, and return the index
    index_shaolong = search(PN)
    #if not found
    if (index_shaolong.values.size == 0):
        #do nothing
        continue
    #if PN found, update SCBOM with Shaolong's BOM info
    else:
        #copy shaolong's BOM info to SCBOM
        copy_and_paste_row(SCBOM, index, SCBOM_shaolong, index_shaolong)


        
            
        
        
    
    

In [31]:
SCBOM.loc[[21]]

,Identifier,Title,Revision,Description,QTY,UOM,Purchased Part Type,Maturity,Part Type,System,...,Phase 1 PSW Date,Phase 3 PSW Date,Comments,Engineering Release Estimtae,Engineering Release Actual,Assembly Station,Part Creation Date,Last Modified Date,Part Status,Part Active
21,1|2|1|1|1|4|1,AA-YF000148NN-00,A.1,HEXAGONAL RIVET NUT M6,1.0,EA EACH,Buy,Released,Part,Y Fasteners,...,,,,NaN,NaN,NaN,NaN,NaN,Latest revision,Active


In [32]:
save(SCBOM)

## Create a new jupyter notebook for this step
### Step 2: Maintain and update SCBOM using updated EBOM

We need to maintain the integrity of the EBOM, so everytime EBOM is updated, a new corresponding supply chain BOM is created with identifical information copied from EBOM. Then the supply chain BOM is updated with information from previous supply chain BOM, parts info whose PN and Rev are identical with EBOM will be copy and pasted in, parts that are no longer identifical will be marked as inactive and/or removed parts. 

In [20]:
import pandas as pd
import numpy as np
import xlrd, openpyxl
import csv, datetime
import time

In [2]:
EBOM_PATH = './Data/AA-ZZ000001NN-03_VP BoM_9-21-18 at 3.42 PM.xlsx'
SCBOM_PATH = './Data/Supply Chain BOM V2 (version 1).xlsx'

In [3]:
#load EBOM
def load():
    # load Engineering BOM
    EBOM = pd.read_excel(EBOM_PATH, sheet_name="BOM", skiprows=[0])
    EBOM.columns = EBOM.columns.str.replace("\(R\)\ ", "") # trim (R) away from the header
    EBOM = EBOM.reset_index(drop=True)
    # load Supply Chain BOM
    SCBOM = pd.read_excel(SCBOM_PATH, sheet_name="Supply Chain BOM")
    SCBOM = SCBOM.reset_index(drop=True)
    return EBOM, SCBOM

EBOM, SCBOM = load()

In [4]:
SCBOM_columns = SCBOM.columns.tolist()
SCBOM_columns_size = len(SCBOM_columns)
EBOM_columns = EBOM.columns.tolist()
EBOM_columns_size = len(EBOM_columns)

In [5]:
print("EBOM shape: ", EBOM.shape)
print("SCBOM shape: ", SCBOM.shape)
print("# of columns in EBOM: ", EBOM_columns_size)
print("# of columns in SCBOM: ", SCBOM_columns_size)

EBOM shape:  (4570, 15)
SCBOM shape:  (3164, 78)
# of columns in EBOM:  15
# of columns in SCBOM:  78


In [6]:
# create a updated SCBOM with columns from SCBOM and data from EBOM
SCBOM_updated = EBOM.copy()
for each in SCBOM.columns.tolist()[15:]:
    SCBOM_updated[each] = ""
SCBOM_updated.shape

(4570, 78)

In [7]:
# search PN and Rev in SCBOM
def search(df, PN, Rev):
    df = df.loc[(df["Title"]==PN) & (df["Revision"]==Rev)]
    index = df.index.values
    
    # only one entry found
    if (len(index)==1):
        return index   
    
    # multiple entries found
    elif (len(index)>1):
        #return the first index if duplicate PN and Rev found
        return index.tolist()   # returned indexes are a numpy.ndarray, we convert to a list and get the first value
    
    # no entry found
    else:
        return None
        

In [8]:
print(search(EBOM, 'AA-MB001705NN-00', 'D.1'))
print(len(search(EBOM, 'AA-MB001705NN-00', 'D.1')))

[4502]
1


In [9]:
print(search(EBOM, 'AA-MB001703NN-00', 'B.1'))
print(len(search(EBOM, 'AA-MB001703NN-00', 'B.1'))

SyntaxError: unexpected EOF while parsing (<ipython-input-9-34332973a15c>, line 2)

In [10]:
# def replace_bracket(data):
#     value = str(data).replace('[', '').replace(']', '').replace('\'', '')
#     if value == 'nan':
#         return ""
#     return value

# def change_datetime_to_string(data):
#     if data.size == 0:
#         return ""
#     elif (data == np.array([' '])):
#         return ""
#     elif data == "nan":
#         return ""
#     else:
#         return data[0]
    
# #     if data == np.array([' ']):
# #         return ""
# #     if data == "nan":
# #         return ""
# #     else:
# #         return data[0].strftime('%m/%d/%Y')

In [11]:
# # copy EBOM info and paste into SCBOM
# def copy_and_paste_row(df1, index1, df2, index2):
#     # copy df2 info into df1
#     df1.loc[index1,["Identifier"]] = replace_bracket(df2.loc[index2,["Identifier"]].values)
#     df1.loc[index1,["Title"]] = replace_bracket(df2.loc[index2,["Title"]].values)
#     df1.loc[index1,["Revision"]] = replace_bracket(df2.loc[index2,["Revision"]].values)
#     df1.loc[index1,["Description"]] = replace_bracket(df2.loc[index2,["Description"]].values)
#     df1.loc[index1,["QTY"]] = replace_bracket(df2.loc[index2,["QTY"]].values)
#     df1.loc[index1,["UOM"]] = replace_bracket(df2.loc[index2,["UOM"]].values)
#     df1.loc[index1,["Purchased Part Type"]] = replace_bracket(df2.loc[index2,["Purchased Part Type"]].values)
#     df1.loc[index1,["UOM"]] = replace_bracket(df2.loc[index2,["UOM"]].values)
#     df1.loc[index1,["Maturity"]] = replace_bracket(df2.loc[index2,["Maturity"]].values)
#     df1.loc[index1,["Part Type"]] = replace_bracket(df2.loc[index2,["Part Type"]].values)
#     df1.loc[index1,["System"]] = replace_bracket(df2.loc[index2,["System"]].values)
#     df1.loc[index1,["SubSystem"]] = replace_bracket(df2.loc[index2,["SubSystem"]].values)
#     df1.loc[index1,["Legacy Part Number"]] = replace_bracket(df2.loc[index2,["Legacy Part Number"]].values)
#     df1.loc[index1,["Legacy Part Revision"]] = replace_bracket(df2.loc[index2,["Legacy Part Revision"]].values)
#     df1.loc[index1,["Configuration"]] = replace_bracket(df2.loc[index2,["Configuration"]].values)

#     return df1

In [10]:
# copy EBOM info and paste into SCBOM
def copy_and_paste_row(df1, index1, df2, index2):  # index1 and index2 are int
    # copy df2 info into df1
    offset = df2.shape[1]
    for i in range(offset-1, len(df2.columns.tolist()), 1):
        df1.loc[index1, df1.columns.tolist()[i]] = df2.loc[index2, df2.columns.tolist()[i]]
        
    return df1

In [11]:
# save dataframe to Excel
def save(df):
    #https://stackoverflow.com/questions/28837057/pandas-writing-an-excel-file-containing-unicode-illegalcharactererror
    df = df.applymap(lambda x: x.encode('unicode_escape').
                 decode('utf-8') if isinstance(x, str) else x)

    writer = pd.ExcelWriter('Updated Supply Chain BOM.xlsx')
    df.to_excel(writer, sheet_name="Updated Supply Chain BOM", na_rep="" )
    writer.save()

#     df.to_csv("Supply Chain BOM.csv",quoting=csv.QUOTE_NONE, escapechar="\\")

In [52]:
# load EBOM and SCBOM
EBOM, SCBOM = load()

# create a updated SCBOM with columns from SCBOM and data from EBOM
SCBOM_updated = EBOM.copy()
for each in SCBOM.columns.tolist()[14:]:
    SCBOM_updated[each] = ""
SCBOM_updated.shape



(4570, 79)

In [13]:
EBOM.columns

Index(['Identifier', 'Title', 'Revision', 'Description', 'QTY', 'UOM',
       'Purchased Part Type', 'Maturity', 'Part Type', 'System', 'SubSystem',
       'Type', 'Legacy Part Number', 'Legacy Part Revision', 'Configuration'],
      dtype='object')

In [14]:
SCBOM.columns.tolist()[14:]

['Commodity Package',
 'Engineer',
 'GSM',
 'Senior GSM',
 'Supplier Name',
 'Sourcing Status',
 'Active/Inactive',
 'Supplier Contact Person',
 'Supplier Contact Email',
 'Supplier Manufacturing Country',
 'Supplier Manufacturing City',
 'Incoterm',
 'GTC(Y/N)',
 'LOI(Y/N)',
 'Purchase Contract(Y/N)',
 'JIS(Y/N)',
 'Tier',
 'Ship to',
 'Daily Production Capability',
 'Order Lead Time ',
 'MOQ',
 'Purchae Requisition',
 'Purchase Order',
 'First Pickup Date',
 'Last Pickup Date',
 'Unit Price',
 'Tooling Cost',
 'ED&T',
 'Oversea Pick Up Location',
 'Package Cost Estimate',
 'Transport Price quoted?',
 'Total Investment',
 'Origin Currency',
 'HS code Export',
 'HS code Import',
 'Regulatory Requirements (Y/N)',
 'Logistic Planned (Y/N)',
 'Packaging Proposal Approved(Y/N)',
 'EDI Setup',
 'Duties',
 'Taxes',
 'SQE',
 'Priority(H/M/L)',
 'APQP Kick Off Date',
 'APQP Status',
 'Tooling Lead Time',
 'Tooling Kick Off Date',
 'Tooling Completion Date',
 'Home Line',
 'FTN',
 'Grain',
 'AA

In [42]:
# loop through SCBOM 
for index, row in SCBOM.iterrows():
    PN = row["Title"]
    Rev = row["Revision"]
#     Identifier = row["Identifier"] # same part could be structured differently but the part is the same, so if same part found at multiple places, we will just pick one

    # search if this part exists in SCBOM_updated, return the index in SCBOM_updated
    index_SCBOM_updated = search(SCBOM_updated, PN, Rev)
    print(index, index_SCBOM_updated)

0 [1]
1 [2]
2 None
3 [5]
4 [6]
5 [7]
6 [11]
7 [12]
8 [13]
9 [14]
10 [17]
11 [18]
12 [21]
13 [23]
14 [25]
15 [26]
16 [27]
17 [28]
18 [30]
19 [31]
20 [35]
21 [37]
22 [40]
23 [41]
24 [42, 60]
25 [43]
26 [50]
27 [44]
28 [45]
29 [46]
30 [51]
31 [52]
32 [54]
33 [55]
34 [58]
35 [59]
36 [61]
37 None
38 [63]
39 [66]
40 [71]
41 [72]
42 [73]
43 [74]
44 [68]
45 [69]
46 [70]
47 [65]
48 [76]
49 [78]
50 [79]
51 [80]
52 [82]
53 [84]
54 [85]
55 None
56 None
57 [88]
58 [89]
59 [91]
60 [93]
61 [97]
62 [99]
63 [100]
64 [101]
65 [103]
66 [104]
67 [331]
68 [332]
69 [106]
70 [107]
71 [110]
72 [112]
73 [113]
74 [116]
75 [118]
76 [119]
77 [120]
78 [121]
79 [96, 123, 204, 226]
80 [126]
81 [128]
82 [130]
83 [131]
84 [133]
85 None
86 [134]
87 [135, 153, 238, 257]
88 [138]
89 [140]
90 [141]
91 [145]
92 [147]
93 [159]
94 [162]
95 [148]
96 [149]
97 [150]
98 [151]
99 [152]
100 [95]
101 [165]
102 [170]
103 [177]
104 [178]
105 [172]
106 [173]
107 [174]
108 [168, 266]
109 [169, 267]
110 [167]
111 [181]
112 [185]
113 [18

898 [1516, 1530, 1546, 1557, 1719, 1730, 1745, 1758]
899 None
900 [1519, 1535]
901 [1520, 1536]
902 None
903 None
904 [1523, 1538]
905 [1524, 1537]
906 [1540]
907 [1541]
908 [1542]
909 [1549, 1560]
910 [1550, 1561]
911 [1414, 1507]
912 [1417]
913 [1418]
914 [1420]
915 [1421]
916 [1423]
917 [1424]
918 None
919 [1426]
920 [1427, 1506]
921 [1431]
922 [1432]
923 [1433]
924 [1434]
925 [1441]
926 [1442]
927 [1443]
928 [1444]
929 [1445]
930 [1446]
931 [1449]
932 [1450]
933 [1451]
934 [1452]
935 [1457]
936 [1458]
937 [1460]
938 [1461]
939 [1462]
940 [1467]
941 [1468]
942 [1469]
943 [1470]
944 [1471]
945 [1472]
946 [1480]
947 [1481]
948 [1483]
949 [1484]
950 [1435]
951 [1436]
952 [1437]
953 [1438]
954 [1439]
955 [1474]
956 [1476]
957 None
958 None
959 [1486]
960 [1488]
961 [1489]
962 [1490]
963 [1493]
964 [1525]
965 None
966 [1527, 1727]
967 None
968 None
969 None
970 [1551]
971 [1552]
972 [1553]
973 [1494]
974 [1495]
975 [1497]
976 [1498]
977 [1501]
978 [1505]
979 [1502]
980 [1562]
981 [1563]


1619 [3104]
1620 [3105]
1621 [3106]
1622 [3108]
1623 [3110]
1624 [3111]
1625 [3112]
1626 [3114]
1627 [3116]
1628 [3117]
1629 [3118]
1630 [3120]
1631 [3122]
1632 [3123]
1633 [3125]
1634 [3127]
1635 [3128]
1636 [3129]
1637 [3130]
1638 [3131]
1639 [3132]
1640 [3133]
1641 [3134]
1642 [3135]
1643 [3137]
1644 [3138]
1645 [3139]
1646 [3140]
1647 [3141]
1648 [3142]
1649 [3143]
1650 [3145]
1651 [3146]
1652 [3147]
1653 [3148]
1654 [3150]
1655 [3152]
1656 [3153]
1657 [3154]
1658 [3155]
1659 [3157]
1660 [3159]
1661 [3160]
1662 [3161]
1663 [3162]
1664 [3163]
1665 [3164]
1666 [3166]
1667 [3167]
1668 [3168]
1669 [3169]
1670 [3170]
1671 [3171]
1672 None
1673 None
1674 None
1675 None
1676 [3173]
1677 [3174]
1678 [3176]
1679 [3177]
1680 [3179]
1681 [3180]
1682 None
1683 [3184]
1684 [3185]
1685 None
1686 [3189]
1687 [3190]
1688 [3192]
1689 [3194]
1690 [3195]
1691 [3196]
1692 [3197]
1693 None
1694 [3200]
1695 [3201]
1696 [3202]
1697 [3203]
1698 [3204]
1699 [3205]
1700 [3206]
1701 [3207]
1702 [3208]
1703 [

2381 [4420]
2382 [4421]
2383 [4422]
2384 [4423]
2385 [4424]
2386 [4425]
2387 None
2388 None
2389 None
2390 None
2391 None
2392 [4428]
2393 [4429]
2394 [4430]
2395 [4431]
2396 [4444, 4565]
2397 [4492]
2398 [4493]
2399 [4494]
2400 [4495]
2401 [4496]
2402 [4499]
2403 [4500]
2404 [4501]
2405 [4502]
2406 [4503, 4507, 4511, 4515, 4519, 4523, 4527]
2407 [4504, 4508, 4512, 4516, 4520, 4524, 4528]
2408 [4505]
2409 [4506]
2410 [4509]
2411 [4510]
2412 [4513]
2413 [4514]
2414 [4517]
2415 [4518]
2416 [4521]
2417 [4522]
2418 [4525]
2419 [4526]
2420 [4529]
2421 [4530]
2422 [4531]
2423 [4532]
2424 [4533]
2425 [4534]
2426 [4535]
2427 [4536]
2428 [4537]
2429 [4540]
2430 [4541]
2431 [4542]
2432 [4545]
2433 [4546]
2434 [4547]
2435 [4548]
2436 [4549]
2437 [4550]
2438 [4551]
2439 [4552]
2440 [4553]
2441 [4554]
2442 [4555]
2443 [4556]
2444 [4557]
2445 [4559]
2446 [4560]
2447 [4561]
2448 [4562]
2449 [4563]
2450 [4564]
2451 [4479]
2452 [4438]
2453 [4439]
2454 [4440]
2455 [4441]
2456 [4442]
2457 [4443]
2458 [44

3019 None
3020 None
3021 None
3022 None
3023 None
3024 None
3025 None
3026 None
3027 None
3028 None
3029 None
3030 None
3031 None
3032 None
3033 None
3034 None
3035 None
3036 None
3037 None
3038 None
3039 None
3040 None
3041 None
3042 None
3043 None
3044 None
3045 None
3046 None
3047 None
3048 None
3049 None
3050 None
3051 None
3052 None
3053 None
3054 None
3055 None
3056 None
3057 None
3058 None
3059 None
3060 None
3061 None
3062 None
3063 None
3064 None
3065 None
3066 None
3067 None
3068 None
3069 None
3070 None
3071 None
3072 None
3073 None
3074 None
3075 None
3076 None
3077 None
3078 None
3079 None
3080 None
3081 None
3082 None
3083 None
3084 None
3085 None
3086 None
3087 None
3088 None
3089 None
3090 None
3091 None
3092 None
3093 None
3094 None
3095 None
3096 None
3097 None
3098 None
3099 None
3100 None
3101 None
3102 None
3103 None
3104 None
3105 None
3106 None
3107 None
3108 None
3109 None
3110 None
3111 None
3112 None
3113 None
3114 None
3115 None
3116 None
3117 None
3118 None


In [16]:
a = pd.DataFrame({"A": [1,2,3], "B": [4,5,6]})
b = pd.DataFrame({"A": [7,8,9], "B": [0,1,2]})
# a.append(b.loc[0], ignore_index=True)
b.loc[0, "B"] = 1
b.loc[0]
a.append(b.loc[0], ignore_index=True)



,A,B
0,1,4
1,2,5
2,3,6
3,7,1


In [53]:
# loop through SCBOM 

start_time = time.time()

# count how many new parts are added
# how many old parts are removed
removed_parts_count = 0
same_parts = 0

for index, row in SCBOM.iterrows():
    PN = row["Title"]
    Rev = row["Revision"]
    # same part could be structured differently but the part is the same, 
    # so if same part found at multiple places, we will just pick one
#     Identifier = row["Identifier"] 

    # search if this part exists in SCBOM_updated, return the index in SCBOM_updated
    # search() returned a list or None type
    index_SCBOM_updated = search(SCBOM_updated, PN, Rev)


    #not found, deactivate the part, then append this part to SCBOM_updated
    if (index_SCBOM_updated == None):
        SCBOM.loc[index, ["Part Active"]] = "Inactivate"
        SCBOM.loc[index, ["Part Status"]] = "Removed"
        SCBOM.loc[index, ["Part Creation Date"]] = datetime.date(2019, 9, 14)
        SCBOM.loc[index, ["Last Modified Date"]] = datetime.date(2019, 9, 28)
        SCBOM_updated = SCBOM_updated.append(SCBOM.loc[index], ignore_index=True)
        removed_parts_count = removed_parts_count + 1
        


    # found one entry, copy the information to updated SCBOM    
    elif (len(index_SCBOM_updated) == 1):
        index_SCBOM_updated = index_SCBOM_updated[0]
        SCBOM_updated = copy_and_paste_row(SCBOM_updated, index_SCBOM_updated, SCBOM, index)
        SCBOM_updated.loc[index_SCBOM_updated, ["Part Creation Date"]] = datetime.date(2019, 9, 21)
        SCBOM_updated.loc[index_SCBOM_updated, ["Part Status"]] = "Lateste Revision"
        SCBOM_updated.loc[index_SCBOM_updated, ["Part Active"]] = "Active"
        same_parts = same_parts + 1

    # found multiple entries, copy the information to each of the entries in SCBOM_updated
    else:
        for each in index_SCBOM_updated:
            SCBOM_updated = copy_and_paste_row(SCBOM_updated, each, SCBOM, index)
            SCBOM_updated.loc[index_SCBOM_updated, ["Part Creation Date"]] = datetime.date(2019, 9, 21)
            SCBOM_updated.loc[index_SCBOM_updated, ["Part Status"]] = "Lateste Revision"
            SCBOM_updated.loc[index_SCBOM_updated, ["Part Active"]] = "Active"
            same_parts = same_parts + 1


save(SCBOM_updated)

print("\nAfter Sync")
print("EBOM shape: ", EBOM.shape)
print("SCBOM shape: ", SCBOM.shape)
print("SCBOM_updated ", SCBOM_updated.shape)

# num_of_parts_added = len(SCBOM_updated[SCBOM_updated["Part Creation Date"]=='']) 
# num_of_parts_removed = SCBOM_updated.shape[0] - EBOM.shape[0]
# print("\n# of new parts added: ", num_of_parts_added)
# print("# of old parts removewd: ", num_of_parts_removed)

print("\n# of new parts added: ", EBOM.shape[0]-same_parts)
print("# of old parts removewd: ", removed_parts_count)

execution_time = time.time() - start_time
print("\nThis script took --- {} seconds ---".format(execution_time))


After Sync
EBOM shape:  (4570, 15)
SCBOM shape:  (3164, 78)
SCBOM_updated  (5423, 79)

# of new parts added:  1508
# of old parts removewd:  853


This script took --- 39.83958721160889 seconds ---
